In [17]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [18]:
# Loading the data from the Excel spreadsheet
monthly_data = pd.read_excel('../Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Monthly', header=1)

# Grouping by 'Month' and 'Brand', and summing the 'Revenue'
monthly_grouped = monthly_data.groupby(['Month', 'Brand'])['Revenue'].sum().reset_index()

# Plotting the monthly revenue trends by brand
fig = px.line(monthly_grouped, x='Month', y='Revenue', color='Brand', title='Monthly Revenue Trends by Brand',
              labels={'Month': 'Month', 'Revenue': 'Revenue ($)'})

# Show the plot
fig.show()


Average Price Trending Over Time

In [19]:
# Load data from the 'Weekly' sheet
weekly_data = pd.read_excel('../Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Weekly', header=1)

# Convert 'Week' to datetime (adjust the string split if the date format is different)
weekly_data['Week'] = pd.to_datetime(weekly_data['Week'].str.split(' - ').str[0])

# Group by 'Week' and calculate the average price
weekly_avg_price = weekly_data.groupby('Week')['Price'].mean().reset_index()

# Create the line chart
fig = px.line(weekly_avg_price, x='Week', y='Price', title='Average Price Trending Over Time')

# Improve the layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Average Price ($)',
    xaxis=dict(
        tickmode='auto',  # you can change to 'linear' if you want even spacing regardless of the actual time gaps
        nticks=20  # Adjust the number of ticks to get a clear view of the data points
    )
)

# Show the figure
fig.show()

Weekly & Monthly Revenue Trends (Whole market)

Theres a huge dip in march, but interestingly it only shows the sharp dip on the weekly chart. the monthly chart tells a different story. However, the last month shows significantly lower sales, likely due to a recall. 


In [20]:
# Load data from the Weekly sheet
weekly_data = pd.read_excel('../Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Weekly', header=1)

# Load data from the Monthly sheet
monthly_data = pd.read_excel('../Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Monthly', header=1)

# Processing for weekly_data
weekly_data['Week'] = weekly_data['Week'].str.split(' - ').str[0].str.strip()
weekly_data['Week'] = pd.to_datetime(weekly_data['Week'], errors='coerce')
weekly_data.sort_values('Week', inplace=True)
weekly_revenue = weekly_data.groupby('Week')['Revenue'].sum().reset_index()

# Processing for monthly_data
# Combining 'Year' and 'Month' to form a datetime object
monthly_data['Month'] = pd.to_datetime(monthly_data['Year'].astype(str) + '-' + monthly_data['Month'], format='%Y-%B', errors='coerce')
monthly_data.sort_values('Month', inplace=True)
monthly_revenue = monthly_data.groupby('Month')['Revenue'].sum().reset_index()

# Plotting weekly revenue trends
fig_weekly = px.line(weekly_revenue, x='Week', y='Revenue',
              title='Time Series Analysis of Weekly Revenue Trends',
              labels={'Revenue': 'Total Revenue', 'Week': 'Date'})

fig_weekly.update_layout(xaxis_title='Date', yaxis_title='Revenue ($)')
fig_weekly.show()

# Plotting monthly revenue trends
fig_monthly = px.line(monthly_revenue, x='Month', y='Revenue',
              title='Time Series Analysis of Monthly Revenue Trends',
              labels={'Revenue': 'Total Revenue', 'Month': 'Date'})

fig_monthly.update_layout(xaxis_title='Date', yaxis_title='Revenue ($)')
fig_monthly.show()

Segment Trends. aggregated daily data for all segment products. There is a huge dip in march that should probably be investigated. 

Volume is very steady overall, however, overall revenue over the year has trended down. Perhaps food is being sourced less overall on amazon. 

In [22]:
# Load the data
data = pd.read_excel('../Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Segment Trends', header=1)

# Make sure the 'Date' column is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Initialize a figure
fig = go.Figure()

# Add traces for unit sales and revenue
fig.add_trace(go.Scatter(x=data['Date'], y=data['Unit Sales'], mode='lines+markers', name='Unit Sales'))
fig.add_trace(go.Scatter(x=data['Date'], y=data['Revenue'], mode='lines+markers', name='Revenue'))

# Customize the layout
fig.update_layout(
    title='Unit Sales and Revenue Over Time',
    xaxis_title='Date',
    yaxis_title='Count / Amount',
    legend_title='Metric'
)

# Show the plot
fig.show()

Unit Sales Growth Trend, Monthly

seems there was a recall in december 2023 which sharply affected brand sales. https://www.petful.com/brands/purina-recall/

It seems some of the same sources of ingredients may have been in multiple brands. These dropped down to 0 in December 2023. 

1. Hoki Found

2. Reopet

3. Wellness Natural Pet Food

4. Petlibro

5.Hill'S Science Diet

6.Hill'S Prescription Diet

7.Purina Pro Plan

8.Purina Pro Plan

9.Purina Fancy Feast


SQUISHMALLOW had massive growth; and interestingly they seem to be plushies for humans. Seems they have had big success in this segment as well. 

In [ ]:
# Load the monthly data
monthly_data = pd.read_excel('../Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Monthly', header=1)

# Convert the 'Month' column to datetime to ensure proper sorting
monthly_data['Month'] = pd.to_datetime(monthly_data['Month'], format='%B')

# Sort the data
monthly_data.sort_values(by=['Brand', 'Year', 'Month'], inplace=True)

# Group by Brand and calculate the first and last entry for Units
# Assuming that the data is structured such that the first entry is from January and the last from December
brand_growth = monthly_data.groupby('Brand').agg(
    first_units=pd.NamedAgg(column='Units', aggfunc='first'),
    last_units=pd.NamedAgg(column='Units', aggfunc='last')
).reset_index()

def calculate_growth(row):
    if row['first_units'] == 0:
        return None if row['last_units'] == 0 else float('inf')
    return ((row['last_units'] - row['first_units']) / row['first_units']) * 100

brand_growth['Growth'] = brand_growth.apply(calculate_growth, axis=1)

# Calculate the growth percentage

# Sort by growth percentage
brand_growth = brand_growth.sort_values(by='Growth', ascending=False)

# Create a bar chart
fig = px.bar(brand_growth, x='Brand', y='Growth', title='Year-Over-Year Unit Sales Growth by Brand')

# Show the figure
fig.show()
